<img src="./images/DLI_Header.png" style="width: 400px;">

# 預先訓練的模型

儘管通常會需要使用大型且經過完整註釋的資料集來解決深度學習難題，但我們可以直接使用很多免費提供的預先訓練模型。當你決定要展開自己的深度學習專案時，在網路上尋找有助於達成目標的現有模型，會是很好的起步方法。最適合探索可用模型的地方就是 [NGC](https://ngc.nvidia.com/catalog/models)。GitHub 上也有許多可透過 Google 搜尋找到的模型。

## 目標

* 使用 Keras 載入訓練成果良好的預先訓練模型
* 預先處理自己的影像以便用於預先訓練的模型
* 使用預先訓練模型對影像執行精確推論

## 自動化狗門

在本節中，我們要建立只允許狗 (而且禁止其他動物) 進出的狗門。我們可以確保貓留在屋內，而其他動物待在屬於牠們的戶外空間。我們會使用目前提到的各種技術，因此需要非常大型的資料集，其中包含許多狗以及其他動物的圖片。幸運的是，有個容易取得的模型已經過大型資料集的訓練，其中包括各式各樣的動物。

[ImageNet 挑戰](https://en.wikipedia.org/wiki/ImageNet#History_of_the_ImageNet_challenge)催生出許多可用於影像分類的先進模型。這些模型經過數百萬張影像的訓練，並且可以將影像準確分類成 1000 種不同的類別。其中許多類別都是動物，包括不同品種的狗和貓。這就是最適合這個狗門的完美模型。

## 載入模型

首先我們要下載模型。經過訓練的 ImageNet 模型可以直接在 Keras 函式庫中下載。你可以[在這裡](https://keras.io/api/applications/#available-models)看到可用模型及其詳細資訊。其中任何一個模型都可以用在我們的練習。我們要挑選一個常用的模型，叫做 [VGG16](https://keras.io/api/applications/vgg/)：

In [ ]:
from tensorflow.keras.applications import VGG16
  
# load the VGG16 network *pre-trained* on the ImageNet dataset
model = VGG16(weights="imagenet")

現在載入完成了，讓我們來看看模型。看起來和我們在手語練習中用的卷積模型很類似。請注意第一層 (輸入層) 和最後一層 (輸出層)。與先前的練習一樣，我們必須確保影像符合模型預期的輸入維度。瞭解模型會從最終輸出層回傳什麼結果，也是很重要的一步。

In [ ]:
model.summary()

### 輸入維度
我們可以看到模型預期的影像是 (224, 224, 3)，也就是 224 像素高、224 像素寬和 3 個色頻的影像。正如我們在上一個練習中所學到的，Keras 模型一次可以接受多張影像來進行預測。如果只傳遞一張影像，形狀會是 (1, 224, 224, 3)。我們必須確保將影像傳遞到模型進行預測時，影像與這些維度相符。

### 輸出維度
我們也可以看出，模型會回傳形狀 1000 的預測結果。請回想一下我們的第一個練習，模型的輸出形狀是 10，也就是有 10 個不同的數字。在我們的第二個練習中，我們的形狀是 24，也就是靜態影像中呈現的手語字母表中的 24 個字母。在這個練習中，我們有 1000 種類別可用來歸類影像。儘管完整的 ImageNet 資料集有超過 20,000 種類別，競賽和最終的預先訓練模型只有使用這些類別的子集，也就是其中的 1000 種。我們可以看看所有的[可能類別，請見此處](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a)。

有許多類別都是動物，包括不同種類的狗和貓。狗的範圍為類別 151 到 268。貓的範圍為類別 281 到 285。我們可以使用這些類別告訴我們的狗門，門前是什麼類型的動物，以及是否應該讓動物進入。

## 載入影像
我們要先載入並顯示影像，就像我們在先前練習中所做的一樣。

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_image(image_path):
    image = mpimg.imread(image_path)
    print(image.shape)
    plt.imshow(image)

In [ ]:
show_image("doggy_door_images/happy_dog.jpg")

## 預先處理影像

接下來，我們要預先處理影像，以便傳遞至模型。這和我們在上一個練習要預測手語影像時所採取的動作一樣。請記得在這個練習中，影像的最終形狀必須是 (1, 224, 224, 3)。

直接使用 Keras 載入模型時，我們也可以利用 [`preprocess_input`方法](https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/preprocess_input)。這些方法與特定模型相關聯，讓使用者可以預先處理自己的影像，以符合模型最初訓練時採用的影像品質。之前在使用新的美國手語影像執行推論時，我們必須手動執行此動作：

In [1]:
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.vgg16 import preprocess_input

def load_and_process_image(image_path):
    # Print image's original shape, for reference
    print('Original image shape: ', mpimg.imread(image_path).shape)
    
    # Load in the image with a target size of 224, 224
    image = image_utils.load_img(image_path, target_size=(224, 224))
    # Convert the image from a PIL format to a numpy array
    image = image_utils.img_to_array(image)
    # Add a dimension for number of images, in our case 1
    image = image.reshape(1,224,224,3)
    # Preprocess image to align with original ImageNet dataset
    image = preprocess_input(image)
    # Print image's shape after processing
    print('Processed image shape: ', image.shape)
    return image

In [ ]:
processed_image = load_and_process_image("doggy_door_images/brown_bear.jpg")

## 進行預測

現在我們的影像已經是正確的格式，我們可以將影像傳遞到模型並進行預測。我們預期的輸出是 1000 元素的陣列，這個結果應該會很難讀懂。所幸直接透過 Keras 載入的模型還有另一種實用的方法，可將預測陣列轉換成更容易讀懂的形式。

請填寫下列函數以實作預測：

In [ ]:
from tensorflow.keras.applications.vgg16 import decode_predictions

def readable_prediction(image_path):
    # Show image
    FIXME
    # Load and pre-process image
    image = FIXME
    # Make predictions
    predictions = FIXME
    # Print predictions in readable form
    print('Predicted:', decode_predictions(predictions, top=3))

### 解決方案

按一下以下的「...」來查看解決方案。

```python
from tensorflow.keras.applications.vgg16 import decode_predictions

def readable_prediction(image_path):
    # Show image
    show_image(image_path)
    # Load and pre-process image
    image = load_and_process_image(image_path)
    # Make predictions
    predictions = model.predict(image)
    # Print predictions in readable form
    print('Predicted:', decode_predictions(predictions, top=3))
```

用幾種動物試試看，然後看看結果如何！也歡迎你上傳自己的影像並分類，看看模型的表現有多好。

In [ ]:
readable_prediction("doggy_door_images/happy_dog.jpg")

In [ ]:
readable_prediction("doggy_door_images/brown_bear.jpg")

In [ ]:
readable_prediction("doggy_door_images/sleepy_cat.jpg")

## 僅限狗

透過我們的模型進行預測之後，我們就可以運用類別僅讓狗進出，同時確保貓留在屋內。別忘了，狗的範圍是類別 151 至 268，貓的範圍是類別 281 至 285。[argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) 函數可以找出哪一個預測陣列元素是首選類別。

In [ ]:
import numpy as np

def doggy_door(image_path):
    show_image(image_path)
    image = load_and_process_image(image_path)
    preds = model.predict(image)
    if FIXME:
        print("Doggy come on in!")
    elif FIXME:
        print("Kitty stay inside!")
    else:
        print("You're not a dog! Stay outside!")

### 解決方案

按一下以下的「...」來查看解決方案。

```python
import numpy as np

def doggy_door(image_path):
    show_image(image_path)
    image = load_and_process_image(image_path)
    preds = model.predict(image)
    if 151 <= np.argmax(preds) <= 268:
        print("Doggy come on in!")
    elif 281 <= np.argmax(preds) <= 285:
        print("Kitty stay inside!")
    else:
        print("You're not a dog! Stay outside!")
```

In [ ]:
doggy_door("doggy_door_images/brown_bear.jpg")

In [ ]:
doggy_door("doggy_door_images/happy_dog.jpg")

In [ ]:
doggy_door("doggy_door_images/sleepy_cat.jpg")

## 摘要

做得好！我們利用功能強大的預先訓練模型，只靠著幾行程式碼就打造出順利運作的狗門。我們希望你有很開心地發現，其實不需要太多前置工作就能充分運用深度學習。最棒的是，隨著深度學習社群的進步，將來會有更多模型可供你應用在自己的專案。

### 清除記憶體
在繼續之前，請執行下列儲存格以清除 GPU 記憶體。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 下一步

使用預先訓練模型的效果出奇地好，但有時這些模型並不完全適合你的資料。在下一節，你會學到另一項強大的技術：*遷移學習*，透過這種方式你就能依照自身需求調整預先訓練的模型，讓模型對你的資料做出準確預測。

請繼續前往下一節：[*預先訓練的模型*](./05b_presidential_doggy_door.ipynb)。